2-How-Does-DGL-Represent-A-Graph

In [2]:
import os
os.environ['DGLBACKEND'] = 'pytorch'
import numpy as np
import torch
import dgl

In [3]:
g = dgl.DGLGraph(([0,0,0,0,0], [1,2,3,4,5]), num_nodes=6)

/home/dloader-ubuntu/.pyenv/versions/3.10.8/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/home/dloader-ubuntu/.pyenv/versions/3.10.8/lib/python3.10/site-packages/dgl/heterograph.py:83: DGLWarning: Keyword arguments ['num_nodes'] are deprecated in v0.5, and can be safely removed in all cases.
  dgl_warning('Keyword arguments {} are deprecated in v0.5, and can be safely'


In [4]:
# Defining a graph using PyTorch LongTensor
h = dgl.graph((torch.LongTensor([0,0,0,0,0]), torch.LongTensor([1,2,3,4,5])), num_nodes=6)

In [5]:
#
print(g.edges())

(tensor([0, 0, 0, 0, 0]), tensor([1, 2, 3, 4, 5]))


 Although the types of node and edge attributes can be arbitrary in real world, DGLGraph only accepts attributes stored in tensors (with numerical contents). Consequently, an attribute of all the nodes or edges must have the same shape. In the context of deep learning, those attributes are often called features.

In [6]:
# Assign a 3-dim node feature vector for each node
g.ndata['x'] = torch.randn(6,3)
# Assign a 4-dim edge feature vector for each edge
g.edata['a'] = torch.randn(5,4)
# Assign a 5x4 node feature matrix for each node. 
g.ndata['y'] = torch.randn(6,5,4)

print(g.edata['a'])

tensor([[-2.1718, -0.8067,  1.6247,  1.4790],
        [ 2.1533,  1.0617,  0.7053, -0.4530],
        [ 1.4646,  1.2775,  1.0784,  0.2207],
        [ 0.3828,  0.0276, -0.8219,  1.2406],
        [-0.3478, -0.1758, -0.6334,  0.0076]])


In [7]:
g

Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5, 4), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})

In [8]:
# Querying Graph Structures
print(f"Number of nodes: {g.num_nodes()} and number of edges: {g.num_edges()}")

Number of nodes: 6 and number of edges: 5


In [9]:
print(f"Out degree of node 0: {g.out_degrees(0)}, In degree of node 0: {g.in_degrees(0)}")

Out degree of node 0: 5, In degree of node 0: 0


In [10]:
# Graph Transformations (Eg for extracting subgraphs)

# Induce a subgraph from node 0, node 1, and node 3 from the original graph, g
sg1 = g.subgraph([0, 1, 3])

In [11]:
sg1

Graph(num_nodes=3, num_edges=2,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5, 4), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [12]:
# Induce an edge subgraph from node 0, node 1, and node 3 from the original graph, g
sg2 = g.edge_subgraph([0,1,3])

In [13]:
sg2

Graph(num_nodes=4, num_edges=3,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5, 4), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [14]:
# The original IDs of each node in sg1
print(sg1.ndata[dgl.NID])
# The original IDs of each edge in sg1
print(sg1.edata[dgl.EID])
# The original IDs of each node in sg2
print(sg2.ndata[dgl.NID])
# The original IDs of each edge in sg2
print(sg2.edata[dgl.EID])

tensor([0, 1, 3])
tensor([0, 2])
tensor([0, 1, 2, 4])
tensor([0, 1, 3])


In [15]:
# The original node feature of each node in sg1
print(sg1.ndata["x"])
# The original edge feature of each node in sg1
print(sg1.edata["a"])
# The original node feature of each node in sg2
print(sg2.ndata["x"])
# The original edge feature of each node in sg2
print(sg2.edata["a"])

tensor([[ 0.3963,  1.1797,  0.2630],
        [-1.0089, -0.2861,  0.1708],
        [-1.0780,  0.1091, -1.1694]])
tensor([[-2.1718, -0.8067,  1.6247,  1.4790],
        [ 1.4646,  1.2775,  1.0784,  0.2207]])
tensor([[ 0.3963,  1.1797,  0.2630],
        [-1.0089, -0.2861,  0.1708],
        [ 0.0830,  2.3827,  0.8988],
        [ 0.9513,  0.3619, -1.1611]])
tensor([[-2.1718, -0.8067,  1.6247,  1.4790],
        [ 2.1533,  1.0617,  0.7053, -0.4530],
        [ 0.3828,  0.0276, -0.8219,  1.2406]])


In [16]:
newg = dgl.add_reverse_edges(g)
print(newg.edges())

(tensor([0, 0, 0, 0, 0, 1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5, 0, 0, 0, 0, 0]))


In [18]:
# Loading and Saving Graphs
dgl.save_graphs("../data/graph.dgl", g)
dgl.save_graphs("../data/graphs.dgl", [g, sg1, sg2])

In [19]:
# Loading graphs 
g1 = dgl.load_graphs("../data/graph.dgl")

In [20]:
g1

([Graph(num_nodes=6, num_edges=5,
        ndata_schemes={'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
        edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})],
 {})

In [24]:
# Loads graphs as tuple of graphs and dict of labels. In this case the label dict is empty
(g2, sg3, sg4), _ = dgl.load_graphs("../data/graphs.dgl")

In [26]:
g2

Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})